In [21]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [22]:
# Read the CSV file from the Resources folder into a Pandas DataFrame

# Specify the file path
file_path = "../Data/prosperLoanData.csv"

In [23]:
# Read the CSV file into a Pandas DataFrame
Uncleaned_df = pd.read_csv(file_path)

# Review the DataFrame

Uncleaned_df.head()

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
0,1021339766868145413AB3B,193129,2007-08-26 19:09:29.263000000,C,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,...,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258
1,10273602499503308B223C1,1209647,2014-02-27 08:28:07.900000000,NaN,36,Current,NaN,0.12016,0.0920,0.0820,...,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
2,0EE9337825851032864889A,81716,2007-01-05 15:00:47.090000000,HR,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,...,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41
3,0EF5356002482715299901A,658116,2012-10-22 11:02:35.010000000,NaN,36,Current,NaN,0.12528,0.0974,0.0874,...,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158
4,0F023589499656230C5E3E2,909464,2013-09-14 18:38:39.097000000,NaN,36,Current,NaN,0.24614,0.2085,0.1985,...,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20


In [24]:
## CLEAN UP DATA 


In [25]:
#types of columns 
specific_columns = ['Term','LoanStatus','ProsperRating (Alpha)','BorrowerState','Occupation','EmploymentStatus','IsBorrowerHomeowner','IncomeRange','EstimatedLoss','LP_GrossPrincipalLoss',
'BorrowerRate','EmploymentStatusDuration','LoanMonthsSinceOrigination','MonthlyLoanPayment','LoanOriginalAmount']  
# List of columns you want to check



In [26]:
Uncleaned_df = Uncleaned_df[specific_columns].copy()
Uncleaned_df.head()

,Term,LoanStatus,ProsperRating (Alpha),BorrowerState,Occupation,EmploymentStatus,IsBorrowerHomeowner,IncomeRange,EstimatedLoss,LP_GrossPrincipalLoss,BorrowerRate,EmploymentStatusDuration,LoanMonthsSinceOrigination,MonthlyLoanPayment,LoanOriginalAmount
0,36,Completed,NaN,CO,Other,Self-employed,True,"$25,000-49,999",NaN,0.0,0.1580,2.0,78,330.43,9425
1,36,Current,A,CO,Professional,Employed,False,"$50,000-74,999",0.0249,0.0,0.0920,44.0,0,318.93,10000
2,36,Completed,NaN,GA,Other,Not available,False,Not displayed,NaN,0.0,0.2750,NaN,86,123.32,3001
3,36,Current,A,GA,Skilled Labor,Employed,True,"$25,000-49,999",0.0249,0.0,0.0974,113.0,16,321.45,10000
4,36,Current,D,MN,Executive,Employed,True,"$100,000+",0.0925,0.0,0.2085,44.0,6,563.97,15000


In [34]:
specific_columns_uncleaned_df = Uncleaned_df[specific_columns]
specific_columns_uncleaned_df.head(20)

,Term,LoanStatus,ProsperRating (Alpha),BorrowerState,Occupation,EmploymentStatus,IsBorrowerHomeowner,IncomeRange,EstimatedLoss,LP_GrossPrincipalLoss,BorrowerRate,EmploymentStatusDuration,LoanMonthsSinceOrigination,MonthlyLoanPayment,LoanOriginalAmount
0,36,Completed,NaN,CO,Other,Self-employed,True,"$25,000-49,999",NaN,0.00,0.1580,2.0,78,330.43,9425
1,36,Current,A,CO,Professional,Employed,False,"$50,000-74,999",0.0249,0.00,0.0920,44.0,0,318.93,10000
2,36,Completed,NaN,GA,Other,Not available,False,Not displayed,NaN,0.00,0.2750,NaN,86,123.32,3001
3,36,Current,A,GA,Skilled Labor,Employed,True,"$25,000-49,999",0.0249,0.00,0.0974,113.0,16,321.45,10000
4,36,Current,D,MN,Executive,Employed,True,"$100,000+",0.0925,0.00,0.2085,44.0,6,563.97,15000
5,60,Current,B,NM,Professional,Employed,True,"$100,000+",0.0449,0.00,0.1314,82.0,3,342.37,15000
6,36,Current,E,KS,Sales - Retail,Employed,False,"$25,000-49,999",0.1275,0.00,0.2712,172.0,11,122.67,3000
7,36,Current,C,CA,Laborer,Employed,False,"$25,000-49,999",0.0799,0.00,0.2019,103.0,10,372.60,10000
8,36,Current,AA,IL,Food Service,Employed,True,"$25,000-49,999",0.0099,0.00,0.0629,269.0,3,305.54,10000
9,36,Current,AA,IL,Food Service,Employed,True,"$25,000-49,999",0.0099,0.00,0.0629,269.0,3,305.54,10000


In [35]:
specific_columns_uncleaned_df.dtypes

Term                            int64
LoanStatus                     object
ProsperRating (Alpha)          object
BorrowerState                  object
Occupation                     object
EmploymentStatus               object
IsBorrowerHomeowner              bool
IncomeRange                    object
EstimatedLoss                 float64
LP_GrossPrincipalLoss         float64
BorrowerRate                  float64
EmploymentStatusDuration      float64
LoanMonthsSinceOrigination      int64
MonthlyLoanPayment            float64
LoanOriginalAmount              int64
dtype: object

In [47]:
specific_numeric_columns = specific_columns_uncleaned_df[['Term','EstimatedLoss','LP_GrossPrincipalLoss','BorrowerRate','EmploymentStatusDuration','LoanMonthsSinceOrigination','MonthlyLoanPayment','LoanOriginalAmount']]
specific_numeric_columns.head()


,Term,EstimatedLoss,LP_GrossPrincipalLoss,BorrowerRate,EmploymentStatusDuration,LoanMonthsSinceOrigination,MonthlyLoanPayment,LoanOriginalAmount
0,36,NaN,0.0,0.1580,2.0,78,330.43,9425
1,36,0.0249,0.0,0.0920,44.0,0,318.93,10000
2,36,NaN,0.0,0.2750,NaN,86,123.32,3001
3,36,0.0249,0.0,0.0974,113.0,16,321.45,10000
4,36,0.0925,0.0,0.2085,44.0,6,563.97,15000


In [53]:
# Initialize the StandardScaler
Scaled_columns = StandardScaler().fit_transform(
    specific_numeric_columns
)

In [54]:
# Fit and transform the selected columns
Scaled_df = pd.DataFrame(Scaled_columns)
Scaled_df.head()

#Columns not included
# 'LoanStatus','ProsperRating (Alpha)','BorrowerState','Occupation','EmploymentStatus','IsBorrowerHomeowner','IncomeRange'



,0,1,2,3,4,5,6,7
0,-0.462837,NaN,-0.293257,-0.464648,-0.995676,1.538101,0.300753,0.174196
1,-0.462837,-1.184816,-0.293257,-1.346787,-0.551138,-1.064150,0.241074,0.266258
2,-0.462837,NaN,-0.293257,1.099145,NaN,1.804998,-0.774043,-0.854340
3,-0.462837,-1.184816,-0.293257,-1.274612,0.179174,-0.530355,0.254151,0.266258
4,-0.462837,0.260763,-0.293257,0.210323,-0.551138,-0.863977,1.512708,1.066799


In [55]:
# Header
# ListingKey
# Term
# LoanStatus
# ProsperRating (Alpha)
# BorrowerState
# Occupation
# EmploymentStatus
# IsBorrowerHomeowner
# IncomeRange
# EstimatedLoss
# LP_GrossPrincipalLoss
# BorrowerRate
# EmploymentStatusDuration
# LoanMonthsSinceOrigination
# MonthlyLoadPayment
# LoanOriginalAmount

NameError: name 'Header' is not defined